# Semantic Kernel を使用したシングルエージェントチャット完了システム

## 概要

このノートブックでは、Semantic Kernel フレームワークを使用してシングルエージェントのチャット完了システムを構築する方法を学習します。

### 主な学習内容

1. **Chat Completion Agent の基本構造**
   - Azure OpenAI Service との接続
   - Semantic Kernel エージェントの作成と設定

2. **Model Context Protocol (MCP) プラグインの統合**
   - 外部データソースへの接続
   - PostgreSQL と CosmosDB を利用したデータアクセス

# ライブラリのインポート

このセクションでは、Semantic Kernel エージェントシステムの構築に必要なライブラリをインポートします。

In [26]:
import os
import json
import datetime

from dotenv import load_dotenv, find_dotenv

from semantic_kernel.agents import (
    ChatCompletionAgent,
    ChatHistoryAgentThread,
)
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin
from semantic_kernel.contents import (
    FunctionCallContent,
    FunctionResultContent,
    TextContent,
)


# 環境変数の取得

Azure OpenAI Service と AI Foundry プロジェクトへの接続に必要な認証情報を環境変数から取得します。

## 必要な環境変数

- **PROJECT_ENDPOINT**: AI Foundry プロジェクトのエンドポイントURL
- **AZURE_DEPLOYMENT_NAME**: デプロイされたモデルの名前
- **AZURE_OPENAI_ENDPOINT**: Azure OpenAI サービスのエンドポイント
- **AZURE_OPENAI_API_KEY**: Azure OpenAI サービスのAPIキー

これらの値は `.env` ファイルに設定して管理します。

In [27]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY=os.getenv("AZURE_OPENAI_API_KEY")

# ユーティリティ関数

エージェントとの対話をデバッグ・監視するためのヘルパー関数を定義します。

### print_thread_message_details()
スレッド内のメッセージ詳細を解析して表示します：
- **Function Call**: エージェントが実行した関数呼び出し
- **Function Result**: 関数実行の結果
- **Text Content**: ユーザーメッセージとエージェントの応答


In [28]:
async def print_thread_message_details(thread: str):
    """
    スレッドのメッセージ詳細を表示します。

    Args:
        thread (str): スレッドのインスタンス
    """
    async for message in thread.get_messages():
        print("-----")

        for item in message.items:
            if isinstance(item, FunctionCallContent):
                print(f"[Function Calling] by {message.ai_model_id}")
                print(f" - Function Name : {item.name}")
                print(f" - Arguments     : {item.arguments}")

            elif isinstance(item, FunctionResultContent):
                print(f"[Function Result]")
                # 文字列のデコード変換
                if isinstance(item.result, str):
                    try:
                        decoded = json.loads(item.result)
                        print(f" - Result        : {decoded}") # デコード成功時は変換後の値を表示
                    except json.JSONDecodeError:
                        print(f" - Result        : {item.result}")  # デコード失敗時はそのまま
                else:
                    print(f" - Result        : {item.result}")

            elif isinstance(item, TextContent):
                if message.name:
                    print(f"[Agent Response] from {message.ai_model_id}")
                else:
                    print("[User Message]")
                print(f" - Content       : {item.text}")

            else:
                print(f"[Unknown Item Type] ({type(item).__name__})")
                print(f" - Raw Item      : {item}")

# プラグインの作成

Model Context Protocol (MCP) プラグインを作成し、外部データソースへの接続を確立し、事前定義されたデータベースクエリの実行します。

### 接続先データソース
- **PostgreSQL**: 企業マスター、注文履歴、ユーザー情報
- **CosmosDB**: 商品レビューと感情分析データ

In [29]:
# MCPのエンドポイントURL
MCP_URL = "http://127.0.0.1:8000/mcp/"


mcp_plugin = MCPStreamableHttpPlugin(
    name="mcp_plugin",
    url=MCP_URL
)

await mcp_plugin.connect()

# クライアントの初期化

Azure OpenAI Service への接続とエージェントクライアントを初期化します。

## 初期化プロセス

1. **Azure Chat Completion Service**: Azure OpenAI への接続クライアント
2. **Chat Completion Agent**: 実際の対話を処理するエージェント

### エージェントの設定

エージェントには以下の特性を設定します：
- **役割**: ECサイトの分析エージェント
- **利用可能ツール**: MCP プラグイン経由でのデータベースアクセス
- **対応データ**: PostgreSQL の各種マスター、注文、ユーザー情報、CosmosDB のレビュー分析データ

In [30]:
# Chat Completion API クライアントの初期化
azure_completion_service  = AzureChatCompletion(
    service_id="azure_completion_agent",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY
)

In [31]:
# Chat Completion Agent クライアントの初期化
agent = ChatCompletionAgent(
    service=azure_completion_service,
    name="mcp_agent",
    instructions=(
        "あなたはECサイトの分析エージェントです。"
        "PostgreSQLの各種マスター、注文、ユーザー情報と、CosmosDBに格納されたレビュー分析データを、mcp_pluginとして利用できます。"
    ),
    plugins=[mcp_plugin],
)

# スレッドの作成

会話履歴を管理するためのスレッドを作成します。


In [32]:
# Thread の作成
thread = ChatHistoryAgentThread()
print(f"Created Thread. THREAD_ID: {thread.id}")


Created Thread. THREAD_ID: thread_eabc1dd8ff5b4d42bbbda35072222563


# レスポンスを取得

エージェントに質問を送信し、回答を取得する実際の対話処理を実行します。

In [33]:
response = await agent.get_response(
    messages=["2024年12月は何がどれくらい購入されましたか？"],
    thread=thread,
)

print(response)

2024年12月に購入された商品の売上は以下の通りです。

- Surface Pro：1,440,000円
- Surface Laptop：170,000円
- Surface Go：90,000円
- Modern Webcam：39,000円
- Office：38,000円
- Xbox Game Pass：1,500円

このように、Surface Proが最も多く購入されています。数量について知りたい場合は、商品ごとの単価等追加情報もご案内できますのでご相談ください。


In [35]:
response = await agent.get_response(
    messages=["2024年1月～12月の月別売上を教えてください"],
    thread=thread,
)

print(response)

2024年1月～12月の月別売上（合計）は以下の通りです。
※「売上」の合計は主な商品の売上金額合算値です。

- 1月：約74万1000円
- 2月：約64万2000円
- 3月：約43万1000円
- 4月：約89万64000円
- 5月：約75万円
- 6月：約28万9000円
- 7月：約81万1000円
- 8月：約43万2000円
- 9月：約41万7000円
- 10月：約16万5000円
- 11月：約18万6000円
- 12月：約1,870,500円

特に12月と4月に売上が大きくなっています。
（もし月ごとに商品構成や詳細リスト等ご要望があれば追加でご案内します）


In [ ]:
await print_thread_message_details(thread)

# 続けて実行

In [36]:
response = await agent.get_response(
    messages=["4月の商品別販売額を教えてください"],
    thread=thread,
)

print(response)

2024年4月の商品別販売額は以下の通りです。

- Surface Laptop：8,500,000円
- Surface Go：180,000円
- Type Cover：40,000円
- USB-C トラベルハブ：33,000円
- Surface ペン：16,000円
- Microsoft 365：16,000円
- Arc Mouse：11,000円
- Xbox Game Pass：4,500円

Surface Laptopが圧倒的に多く販売されています。他商品の詳細も気になる場合はお知らせください。


In [37]:
response = await agent.get_response(
    messages=["4月の商品別販売個数を教えてください"],
    thread=thread,
)

print(response)

「販売個数」を正確にお調べするには商品ごとの単価が必要となります。  
現在分かっているのは4月の商品別売上金額のみです。  
商品ごとの単価データを取得して、個数を計算しましょうか？  
もし既に知っている単価があればご指定いただければ、すぐに計算しご案内できます。


In [38]:
response = await agent.get_response(
    messages=["Surface Laptopの2024年1月から12月の月別販売数量を教えてください"],
    thread=thread,
)

print(response)

各月における「Surface Laptop」の販売数量を計算するため、各月の「Surface Laptop」の売上金額はわかっています。しかし、個数の計算には「Surface Laptop」の単価が必要です。

【2024年のSurface Laptop月別売上（抜粋）】
- 1月：510,000円
- 2月：取扱データなし
- 3月：170,000円
- 4月：8,500,000円
- 5月：データなし
- 6月：データなし
- 7月：170,000円
- 8月：340,000円
- 9月：170,000円
- 10月：データなし
- 11月：データなし
- 12月：170,000円

「Surface Laptop」の1台あたりの販売単価を教えて頂けますか？  
または、商品マスターから単価を取得して個数を計算しましょうか？


In [34]:
await print_thread_message_details(thread)

-----
[User Message]
 - Content       : 2024年12月は何がどれくらい購入されましたか？
-----
[Function Calling] by gpt-4.1
 - Function Name : mcp_plugin-get_sales_by_product
 - Arguments     : {"start_date":"2024-12-01","end_date":"2024-12-31"}
-----
[Function Result]
 - Result        : [TextContent(inner_content=TextContent(type='text', text='[{"product_id": 1, "product_name": "Surface Pro", "total_sales": 1440000}, {"product_id": 2, "product_name": "Surface Laptop", "total_sales": 170000}, {"product_id": 3, "product_name": "Surface Go", "total_sales": 90000}, {"product_id": 8, "product_name": "Modern Webcam", "total_sales": 39000}, {"product_id": 11, "product_name": "Office", "total_sales": 38000}, {"product_id": 15, "product_name": "Xbox Game Pass", "total_sales": 1500}]', annotations=None, meta=None), ai_model_id=None, metadata={}, content_type='text', text='[{"product_id": 1, "product_name": "Surface Pro", "total_sales": 1440000}, {"product_id": 2, "product_name": "Surface Laptop", "total_sales": 1700